## Preprocessing before chuncking

In [55]:
import re 

from bisect import bisect_right

import glob
from typing import List

In [ ]:
def make_knowledge_text(files) -> str:
    entire_knowlegde_base = ""
    with open(files, "r", encoding="utf-8") as f:
        entire_knowlegde_base += f.read()
        entire_knowlegde_base += "\n\n"
        
    return entire_knowlegde_base

In [ ]:
base = "Transformer models.md"
knowledge_text = make_knowledge_text(base)
HEADER_RE = re.compile(r"^(#{1,3})\s+(.+?)\s*$",re.MULTILINE)


In [61]:
def return_knowledge_textlength(knowledge_base: str):
    return len(knowledge_base)


print(
    f"Total character in knowledge_base: {return_knowledge_textlength(knowledge_text)}"
)

Total character in knowledge_base: 13557


In [ ]:
def extract_header_marks(text: str):
    """
    문서를 위에서 아래로 읽으면서, \n
    “현재 내가 속한 #/##/### 섹션 경로”를 계속 업데이트하고, \n
    헤더가 등장하는 위치마다 (그 위치, 그때의 경로)를 기록하는 함수
    """
    header_marks = []
    stack = [None, None, None]  # levels 1..3

    for m in HEADER_RE.finditer(text):
        level = len(m.group(1))
        title = m.group(2).strip()

        stack[level-1] = title
        for i in range(level, 3): stack[i] = None
        path = [x for x in stack if x]
        header_marks.append((m.start(), path))
    return header_marks
extract_header_marks(knowledge_text)



[(592, ['Section1: 환경 설정']),
 (1145, ['Section1: 환경 설정', '필요 라이브러리']),
 (1584, ['Section2: HuggingFace 인증 및 모델 엑세스']),
 (1620, ['Section2: HuggingFace 인증 및 모델 엑세스', 'HuggingFace Hub']),
 (1921, ['Section3: Quantization 이해']),
 (1950, ['Section3: Quantization 이해', '모델 선택']),
 (1969,
  ['see here: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct']),
 (2090, ['see here: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct']),
 (2158, ['LLAMA = "meta-llama/Llama-3.2-1B-Instruct"']),
 (2469, ['Quantization Config - 적은 메모리를 사용해서 모델을 메모리에 load하게 해줌']),
 (3842, ['bfloat16 vs float16 차이:']),
 (3868, ['- 둘 다 16-bit']),
 (3883, ['- bfloat16: 지수부 넓음 → 큰 숫자 표현 좋음 → LLM에 유리']),
 (3927, ['- float16: 정밀도 높음 → 작은 숫자 정확 → 이미지 처리에 유리']),
 (3976, ['Section4: Tokenizer']),
 (4011, ['Section4: Tokenizer', 'Tokenizer?']),
 (4130, ['사람이 보는 것']),
 (4165, ['LLM이 보는 것']),
 (4216, ['Tokenizer의 역할']),
 (4368, ['Tokenizer 로드']),
 (4669, ['테스트 문장']),
 (4709, ['인코딩 (텍스트 → 숫자)']),
 (4842, ['디코딩 (숫자 → 텍스

In [ ]:
def header_path_for_pos(header_marks, pos: int):
    """
    Args:
        - header_marks: list[(mark_pos, path)] sorted by mark_pos
        - pos: chunk start position 
        
    return: path that is in effect at pos (last mark_pos <= pos)
    """
    
    if not header_marks:
        return []

    positions = [p for p, _ in header_marks]
    ins = bisect_right(positions, pos)   # insertion point to the right
    idx = ins - 1

    if idx < 0:
        return []  # pos is before the first header
    return header_marks[idx][1]


[(592, ['Section1: 환경 설정']),
 (1145, ['Section1: 환경 설정', '필요 라이브러리']),
 (1584, ['Section2: HuggingFace 인증 및 모델 엑세스']),
 (1620, ['Section2: HuggingFace 인증 및 모델 엑세스', 'HuggingFace Hub']),
 (1921, ['Section3: Quantization 이해']),
 (1950, ['Section3: Quantization 이해', '모델 선택']),
 (1969,
  ['see here: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct']),
 (2090, ['see here: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct']),
 (2158, ['LLAMA = "meta-llama/Llama-3.2-1B-Instruct"']),
 (2469, ['Quantization Config - 적은 메모리를 사용해서 모델을 메모리에 load하게 해줌']),
 (3842, ['bfloat16 vs float16 차이:']),
 (3868, ['- 둘 다 16-bit']),
 (3883, ['- bfloat16: 지수부 넓음 → 큰 숫자 표현 좋음 → LLM에 유리']),
 (3927, ['- float16: 정밀도 높음 → 작은 숫자 정확 → 이미지 처리에 유리']),
 (3976, ['Section4: Tokenizer']),
 (4011, ['Section4: Tokenizer', 'Tokenizer?']),
 (4130, ['사람이 보는 것']),
 (4165, ['LLM이 보는 것']),
 (4216, ['Tokenizer의 역할']),
 (4368, ['Tokenizer 로드']),
 (4669, ['테스트 문장']),
 (4709, ['인코딩 (텍스트 → 숫자)']),
 (4842, ['디코딩 (숫자 → 텍스

In [66]:
def attach_header_metadata_to_chunks(doc_text: str, chunks):
    """
    Args:
        - doc_text: splitter에 넣은 '그 텍스트'와 동일한 원문 문자열
        - chunks: splitter 결과 (ch.page_content, ch.metadata 존재한다고 가정)
    """
    header_marks = extract_header_marks(doc_text)

    search_from = 0
    for ch in chunks:
        snippet = ch.page_content

        # 1) chunk 텍스트가 원문에서 시작하는 위치 찾기
        pos = doc_text.find(snippet, search_from)
        if pos == -1:
            # 안전장치: 못 찾으면 헤더를 비워두거나 fallback
            ch.metadata["header_path"] = []
            ch.metadata["section"] = ""
            continue

        # 2) 다음 chunk는 이번 chunk 뒤에서부터 찾게 업데이트
        search_from = pos + max(1, len(snippet) // 4)  # 너무 공격적이지 않게

        # 3) 해당 위치의 header_path 찾기
        hp = header_path_for_pos(header_marks, pos)

        # 4) metadata 주입
        ch.metadata["header_path"] = hp
        ch.metadata["section"] = " > ".join(hp) if hp else ""


In [67]:
attach_header_metadata_to_chunks(knowledge_text, [])